In [1]:
""" Starter code for logistic regression model to solve OCR task 
with MNIST in TensorFlow
MNIST dataset: yann.lecun.com/exdb/mnist/
Author: Chip Huyen
Prepared for the class CS 20SI: "TensorFlow for Deep Learning Research"
cs20si.stanford.edu
"""
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

# Define paramaters for the model
learning_rate = 0.01
beta = 0.001
batch_size = 256
n_epochs = 256

In [2]:
# Step 1: Read in data
# using TF Learn's built in function to load MNIST data to the folder data/mnist
mnist = input_data.read_data_sets('/data/mnist', one_hot=True) 

Extracting /data/mnist\train-images-idx3-ubyte.gz
Extracting /data/mnist\train-labels-idx1-ubyte.gz
Extracting /data/mnist\t10k-images-idx3-ubyte.gz
Extracting /data/mnist\t10k-labels-idx1-ubyte.gz


In [3]:
# Step 2: create placeholders for features and labels
# each image in the MNIST data is of shape 28*28 = 784
# therefore, each image is represented with a 1x784 tensor
# there are 10 classes for each image, corresponding to digits 0 - 9. 
# Features are of the type float, and labels are of the type int
X = tf.placeholder(tf.float32, name="Features", shape=(None, 784))
Y = tf.placeholder(tf.int32, name="Labels", shape=(None, 10))

In [5]:
# Step 3: create weights and bias
# weights and biases are initialized to 0
# shape of w depends on the dimension of X and Y so that Y = X * w + b
# shape of b depends on Y
#w = tf.Variable(tf.random_normal(shape=[784, 10], stddev=0.01), name='weights')
#b = tf.Variable(tf.zeros([1, 10]), name="bias")
W1 = tf.get_variable("W1", [784,64], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
b1 = tf.get_variable("b1", [1,64], initializer = tf.zeros_initializer())
W2 = tf.get_variable("W2", [64,32], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
b2 = tf.get_variable("b2", [1,32], initializer = tf.zeros_initializer())
W3 = tf.get_variable("W3", [32,10], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
b3 = tf.get_variable("b3", [1,10], initializer = tf.zeros_initializer())

In [6]:
# Step 4: build model
# the model that returns the logits.
# this logits will be later passed through softmax layer
# to get the probability distribution of possible label of the image
# DO NOT DO SOFTMAX HERE
#Z = tf.add(tf.matmul(X,w),b)
Z1 = tf.add(tf.matmul(X, W1), b1)                                             
A1 = tf.nn.relu(Z1)                                             
Z2 = tf.add(tf.matmul(A1, W2), b2)                                             
A2 = tf.nn.relu(Z2)                                            
Z3 = tf.add(tf.matmul(A2, W3), b3) 

In [9]:
# Step 5: define loss function
# use cross entropy loss of the real labels with the softmax of logits
# use the method:
# tf.nn.softmax_cross_entropy_with_logits(logits, Y)
# then use tf.reduce_mean to get the mean loss of the batch
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Z3, labels=Y))
regularizer1 = tf.nn.l2_loss(W1)
regularizer2 = tf.nn.l2_loss(W2)
regularizer3 = tf.nn.l2_loss(W3)
loss = tf.reduce_mean(loss + beta*(regularizer1 + regularizer2 + regularizer3))

In [15]:
# Step 6: define training op
# using gradient descent to minimize loss
global_step = tf.Variable(0)  # count the number of steps taken.
starter_learning_rate = 0.1
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 32, 0.95, staircase=True)
optimizer = optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss,global_step=global_step)

#optimizer = optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss,global_step=global_step)

In [16]:
with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    n_batches = int(mnist.train.num_examples/batch_size)
    for i in range(n_epochs): # train the model n_epochs times
        total_loss = 0
        for _ in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            # TO-DO: run optimizer + fetch loss_batch
            _,loss_batch = sess.run([optimizer,loss],feed_dict={X:X_batch,Y:Y_batch})
            # 
            total_loss += loss_batch
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))

    print('Total time: {0} seconds'.format(time.time() - start_time))

    print('Optimization Finished!') # should be around 0.35 after 25 epochs

    # test the model
    preds = tf.nn.softmax(Z3)
    correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # need numpy.count_nonzero(boolarr) :(
    
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        accuracy_batch = sess.run([accuracy], feed_dict={X: X_batch, Y:Y_batch}) 
        total_correct_preds += accuracy_batch[0]
    
    print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))

Average loss epoch 0: 0.7728748001227869
Average loss epoch 1: 0.411639711149385
Average loss epoch 2: 0.3568742993836091
Average loss epoch 3: 0.32168084160189764
Average loss epoch 4: 0.2978843773636862
Average loss epoch 5: 0.2800571486075348
Average loss epoch 6: 0.2658345499467627
Average loss epoch 7: 0.255087661478564
Average loss epoch 8: 0.24209190229667682
Average loss epoch 9: 0.23413316153477284
Average loss epoch 10: 0.22536528730225341
Average loss epoch 11: 0.21862419165461977
Average loss epoch 12: 0.21392637017731356
Average loss epoch 13: 0.20693385176290977
Average loss epoch 14: 0.20215969205459702
Average loss epoch 15: 0.19807841113516103
Average loss epoch 16: 0.19419594060316264
Average loss epoch 17: 0.18858105967813563
Average loss epoch 18: 0.18635076249592772
Average loss epoch 19: 0.18374015223757129
Average loss epoch 20: 0.17886569065468333
Average loss epoch 21: 0.17519533564554196
Average loss epoch 22: 0.17491854504447116
Average loss epoch 23: 0.17063

Average loss epoch 190: 0.11588600653911306
Average loss epoch 191: 0.11547177100432253
Average loss epoch 192: 0.11545249962500323
Average loss epoch 193: 0.11540096308861937
Average loss epoch 194: 0.11607970241631303
Average loss epoch 195: 0.11514277933774708
Average loss epoch 196: 0.1159184051103124
Average loss epoch 197: 0.1151922530729637
Average loss epoch 198: 0.11572990099960398
Average loss epoch 199: 0.1149165990684077
Average loss epoch 200: 0.11513451253560102
Average loss epoch 201: 0.11538451857795225
Average loss epoch 202: 0.11517675917282283
Average loss epoch 203: 0.11541998062596143
Average loss epoch 204: 0.11505139236138245
Average loss epoch 205: 0.11483372688711246
Average loss epoch 206: 0.11494860860788933
Average loss epoch 207: 0.11564397693515938
Average loss epoch 208: 0.11501895329941099
Average loss epoch 209: 0.11493429990711614
Average loss epoch 210: 0.1147960953275177
Average loss epoch 211: 0.11537655429026791
Average loss epoch 212: 0.1153327029

In [ ]:
del W1, b1,W2,b2,W3,b3